In [ ]:

#Tutorial Base
#https://stackoverflow.com/questions/73407882/access-in-memory-spark-dataframe-from-different-nodes

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

APP_NAME = "spark.data_processing_engine"
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName(APP_NAME) \
    .getOrCreate()
spark.sparkContext.setLogLevel("WARN")

schemaSnmp = StructType([
    StructField("node-hostname", StringType(), True),
    StructField("event-time", TimestampType(), True),
    StructField("oid", StringType(), True),
    StructField("value", StringType(), True)
])

result = spark.readStream \
    .option("sourceArchiveDir", "/var/datalake-archive") \
    .option("cleanSource", "archive") \
    .schema(schemaSnmp) \
    .json("/var/datalake/snmp-get") \
    .writeStream \
    .queryName("snmpget") \
    .format("parquet") \
    .option("checkpointLocation", "/var/spark-map/snmp-get") \
    .option("path", "/var/spark-map/snmp-get") \
    .start()

result.awaitTermination()
result.stop()

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time 
import datetime 
import logging


APP_NAME = "spark.data_processing_engine"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName(APP_NAME) \
    .getOrCreate()
spark.sparkContext.setLogLevel("WARN")

schema = StructType([
    StructField("node-hostname", StringType(), True),
    StructField("event-time", TimestampType(), True),
    StructField("oid", StringType(), True),
    StructField("value", StringType(), True)
])

results = spark.readStream \
        .schema(schema) \
        .parquet("/var/spark-map/snmp-get/") \
        .writeStream \
        .queryName("snmpget") \
        .format("memory") \
        .start()

i = 0
while i < 60:
    x = spark.table("snmpget").select("value", "oid", "`node-hostname`", "`event-time`").orderBy("`event-time`", ascending=False).head()
    if (x == None):
        print("Data may still be loading, give this approx 30 seconds")
    else: 
        print(f"x = {x}")
    i = i+1
    time.sleep(1)

results.stop()